In [39]:
from sklearn.metrics import mean_absolute_error as MSE
from tensorflow.keras.models import Sequential
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

# Loading our data

In [2]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")

price = df['NO2_price'].values.reshape(-1,1)
fload = df['NO2_load_forecasted'].values.reshape(-1,1)
fgen = df['NO2_generation_forecast'].values.reshape(-1,1)


price_days = seperate_column_to_days(price)
fload_days = seperate_column_to_days(fload)
fgen_days = seperate_column_to_days(fgen)

# Structuring our data

In [3]:
lookbehind = 7
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
stride = 24
hour_in_days = int(no_hours / stride)

price_dataset = []
fload_dataset = []
fgen_dataset = []

for i in range(len(price_days) - hour_in_days+1):
    price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))
    fload_dataset.append(np.concatenate((fload_days[i:i+hour_in_days])))
    fgen_dataset.append(np.concatenate((fgen_days[i:i+hour_in_days])))

price_dataset = np.array(price_dataset)
fload_dataset = np.array(fload_dataset)
fgen_dataset = np.array(fgen_dataset)

scaler = MinMaxScaler()
price_dataset = scaler.fit_transform(price_dataset[:,:,0])
price_dataset = price_dataset[..., np.newaxis].astype(np.float32)

fload_dataset = scaler.fit_transform(fload_dataset[:,:,0])
fload_dataset = fload_dataset[..., np.newaxis].astype(np.float32)

fgen_dataset = scaler.fit_transform(fgen_dataset[:,:,0])
fgen_dataset = fgen_dataset[..., np.newaxis].astype(np.float32)

dataset_forecast = np.concatenate((price_dataset, fload_dataset, fgen_dataset), axis=2)

n,m,k = dataset_forecast.shape

print(dataset_forecast.shape)


(2145, 192, 3)


# Train test split

In [4]:
train = int(0.7*n)
valid = int(0.9*n)
x_train, X_train = price_dataset[:train, :input_width], dataset_forecast[:train, :input_width]
x_valid, X_valid = price_dataset[train:valid, :input_width], dataset_forecast[train:valid, :input_width]
x_test, X_test = price_dataset[valid:, :input_width], dataset_forecast[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset_forecast[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

# Setting up the models

## Hyperparameters

In [51]:
# Common
epochs = 2
loss = "mse" 
optimizer = "adam"
batch_size = [200, 400, 600, 800]
eta_list = np.logspace(-1, -4, 5)

# Univariate
univ_neurons = 128

# Multivariate
multi_neurons_first = 128
multi_neurons_second = 64
filters = 64
kernel_size = 3
strides = 3

# Setting up the univariate SimpleRNN

In [41]:
class RNN(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()

        # RNN architecture
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, 1]))
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)


# return_sequences=True --> (256, 168, 24)
# return_sequences=False --> (256, 24)

## Grid search

In [45]:
columns = ["batch_size", "learning_rate", "epoch", "univ_neurons", "mse_all", "mse_all2"]
columns += [f"mse{i}" for i in range(1,25)]
results = pd.DataFrame(columns=columns)
Y_test_last = Y_test[:,-1]

i = 0
for eta in eta_list[0:2]:
    for batch in batch_size[0:2]:
        RNN_model = RNN(eta, univ_neurons, horizon=horizon)

        # Training
        RNN_model.fit(x_train, Y_train, epochs=epochs, batch_size = batch, validation_data=(x_valid, Y_valid))
        
        # Prediction
        Y_pred_rnn = RNN_model.predict(x_test)
        Y_pred_rnn_last = Y_pred_rnn[:,-1]     
        
        # Results
        mse_val = MSE(Y_test_last, Y_pred_rnn_last)
        se_all = (Y_test_last - Y_pred_rnn_last) ** 2 
        mse_all = np.mean(se_all, axis=0) # The mean of each column 
        mse_val2 = np.mean(mse_all)

        # Storing results
        res1 = np.array([batch, eta, epochs, univ_neurons, mse_val, mse_val2])
        res2 = np.array(mse_all)
        res = np.concatenate([res1,res2], axis=0)
        results.loc[i] = res
        i += 1

results.to_csv(f"{REPORT_DATA}results_best_models_univariate.csv")

Epoch 1/2
8/8 [==============================] - 2s 309ms/step - loss: 12.8995 - val_loss: 11.5794
Epoch 2/2
8/8 [==============================] - 2s 252ms/step - loss: 5.4136 - val_loss: 6.9480
Epoch 1/2
4/4 [==============================] - 1s 364ms/step - loss: 16.6871 - val_loss: 9.8279
Epoch 2/2
4/4 [==============================] - 1s 317ms/step - loss: 11.2391 - val_loss: 4.0163
Epoch 1/2
8/8 [==============================] - 2s 277ms/step - loss: 0.3909 - val_loss: 0.2668
Epoch 2/2
8/8 [==============================] - 2s 243ms/step - loss: 0.1581 - val_loss: 0.0839
Epoch 1/2
4/4 [==============================] - 2s 392ms/step - loss: 0.2991 - val_loss: 0.4601
Epoch 2/2
4/4 [==============================] - 1s 327ms/step - loss: 0.3490 - val_loss: 0.1605


# Setting up the multivariate ConvGRU

In [52]:
class ConvGRU(Sequential):
    def __init__(self, eta, neurons_first, neurons_second, horizon=24):
        super(self.__class__, self).__init__()

        # ConvGRU architecture
        self.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]))
        self.add(keras.layers.GRU(neurons_first, return_sequences=True))
        self.add(keras.layers.GRU(neurons_second, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

In [54]:
columns = ["batch_size", "learning_rate", "epoch", "univ_neurons", "mse_all", "mse_all2"]
columns += [f"mse{i}" for i in range(1,25)]
results = pd.DataFrame(columns=columns)
Y_test_last = Y_test[:,-1]

i = 0
for eta in eta_list[0:1]:
    for batch in batch_size[0:1]:
        ConvGRU_model = ConvGRU(eta, multi_neurons_first, multi_neurons_second, horizon=horizon)

        # Training
        ConvGRU_model.fit(X_train, Y_train[:,2::strides], epochs=epochs, batch_size = batch, validation_data=(X_valid, Y_valid[:,2::strides]))
        
        # Prediction
        Y_pred_rnn = ConvGRU_model.predict(X_test)
        Y_pred_rnn_last = Y_pred_rnn[:,-1]     
        
        # Results
        mse_val = MSE(Y_test_last, Y_pred_rnn_last)
        se_all = (Y_test_last - Y_pred_rnn_last) ** 2 
        mse_all = np.mean(se_all, axis=0) # The mean of each column 
        mse_val2 = np.mean(mse_all)

        # Storing results
        res1 = np.array([batch, eta, epochs, univ_neurons, mse_val, mse_val2])
        res2 = np.array(mse_all)
        res = np.concatenate([res1,res2], axis=0)
        results.loc[i] = res
        i += 1

results.to_csv(f"{REPORT_DATA}results_best_models_multivariate.csv")

Epoch 1/2
8/8 [==============================] - 5s 579ms/step - loss: 1.3856 - val_loss: 0.5181
Epoch 2/2
8/8 [==============================] - 4s 479ms/step - loss: 0.4314 - val_loss: 0.2462
